# Hyperparameter Optimization in PyTorch Lightning with Optuna

In notebook 4 of this exercise, you've learned how to develop and train models with PyTorch Lightning.

In this optional notebook, we'll show you how you can perform hyperparameter optimization in PyTorch Lightning using the  framework *Optuna*.



In [1]:
import shutil
#!pip install pytorch-lightning==0.7.6 > /dev/null
import pytorch_lightning as pl
import torch
from exercise_code.MyPytorchModel import MyPytorchModel
from exercise_code.Util import save_model, load_model, test_and_save

%load_ext autoreload
%autoreload 2

<img src=https://raw.githubusercontent.com/optuna/optuna/master/docs/image/optuna-logo.png></img>

Optuna is an automatic hyperparameter optimization framework, which works with several deep learning libraries, including PyTorch Lightning. Have a look at https://github.com/optuna/optuna!

Two important concepts of Optuna are the terms **Study** and **Trial**:
* **Study**: optimization based on an objective function (e.g.: maximize validation accuracy)
* **Trial**: a single execution of the objective function (i.e., train a model for a specific hyperparameter configuration)

The goal of a study is to find out the optimal set of hyperparameter values through multiple trials (e.g., n_trials=100). Optuna is a framework designed for the automation and the acceleration of the optimization studies.

On a high level, hyper parameter tuning with Optuna works very similar to the search algorithms we implemented in exercise 6. However, Optuna has a set of advantages:

* **Parallelization** of hyperparameter searches over multiple threads or processes without modifying code
* more **efficient search algorithms** for large search spaces and **pruning of unpromising trials** for faster results
* many additional features for automated search of optimal hyperparameters

### Install & Import Optuna

If you haven't done yet, install Optuna via pip by uncommenting the line in the cell below: `!pip install optuna`.

If you want to install Optuna with conda or you have problems with the installation via pip, you can also use `$ conda install -c conda-forge optuna`.

In [2]:
#!pip install optuna
import optuna
from optuna.integration import PyTorchLightningPruningCallback

### Logger

The default logger in PyTorch Lightning automatically writes to event files to be consumed by TensorBoard. 

Here we just set up a simple callback, that keeps the metrics from each validation step in memory, which we can then use to find the best model.

In [3]:
from pytorch_lightning import Callback

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

### Objective

As mentioned above, the goal of our hyperparameter tuning is, to optimize an objective function by finding the best set of hyperparameters. 
(
Optuna is a black-box optimizer, which means we need to provide this objective function (`objective(trial)`), which gets passed a `trial` object and returns a numerical value to evaluate the performance of the current hyperparameters and where to sample in the upcoming trial.

In our case, the metric we want to optimize is the validation accuracy of our models. To get the validation accuracy for a given hyperparameter configuration, we do the same thing as in the previous notebook:

* define a PL-trainer
* define hyper parameters - by sampling them from the provided hyperparameter ranges, similar as in our old random search implementation
* initialize a model with these hyperparameters
* train that model, and report its validation accuracy as our objective function value

As you can see in the code cell below, the implementation is straight forward:

PS: notice the different sampling modes, e.g. `trial.suggest_int`, `trial.suggest_loguniform` which are also similar to our previous implementation! Check out the documentation at https://optuna.readthedocs.io/en/latest/reference/trial.html to find out about further sampling modes, and **feel free to add additional hyperparameters!**

In [4]:
def objective(trial):
    # as explained above, we'll use this callback to collect the validation accuracies
    metrics_callback = MetricsCallback()  
    
    # create a trainer
    trainer = pl.Trainer(
        #train_percent_check=1.0,
        #val_percent_check=1.0,
        logger=False,                                                                  # deactivate PL logging
        max_epochs=30,                                                                  # epochs
        gpus=0 if torch.cuda.is_available() else None,                                 # #gpus
        callbacks=[metrics_callback],                                                  # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_acc"), # early stopping
    )

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"n_hidden": trial.suggest_int("n_hidden", 800, 1200), 
                     "batch_size": 64, 
                     "learning_rate": trial.suggest_loguniform("learning_rate", 5e-3, 5e-2)}
    
    # create model from these hyper params and train it
    model = MyPytorchModel(trial_hparams)
    model.prepare_data()
    trainer.fit(model)

    # save model
    save_model(model, '{}.p'.format(trial.number), "checkpoints")

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["val_acc"]

An important difference to our previous search implementation is, that Optuna does not sample randomly! 

It uses a **Tree-structured Parzen Estimator (TPE)** sampler, which is a kind of bayesian optimization, and more efficient than a pure random search, because it chooses the hyperparameter values after evaluating all previous trials to make a smart guess where the best hyperparameters can be found.

### Run the Search

After we've defined our objective function, we can start the search.

For that, we can also provide a **pruner**, which is a very useful concept supported by Optuna: Similar to *early stopping*, a pruner automatically stops unpromising trials in early stages of training (a.k.a. automated early-stopping) and therefore can significantly speed up the search.

If you want to specify a pruner, have a look at the available options at https://optuna.readthedocs.io/en/latest/reference/pruners.html. 

Then, we create our `study`, define the direction that we want to optimize (i.e., *maximize* the validation accuracy) and start the optimization.

In [5]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=10, timeout=600)

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.
/home/yuxuan/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/yuxuan/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name    | Type        | Params
------------------------------------
0 | model   | Sequential  | 3 M   
1 | model.0 | Linear      | 3 M   
2 | model.1 | BatchNorm1d | 2 K   
3 | model.2 | ReLU        | 0     
4 | model.3 | Linear      | 130 K 
5 | model.4 | BatchNorm1d | 256   
6 | model.5 | ReLU        | 0    

Val-Acc=0.0012


/home/yuxuan/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.4106


Val-Acc=0.4409


...

Val-Acc=0.5167


Val-Acc=0.5176


Val-Acc=0.5135


Val-Acc=0.5135



[I 2020-06-14 21:26:57,944] Finished trial#0 with value: 0.5135 with parameters: {'n_hidden': 1019, 'learning_rate': 0.038747571156293785}. Best is trial#0 with value: 0.5135.


Notice that we save every model in a directory called `./checkpoints`.

### Check the Results

After we've finished the search, we access the best trial at `study.best_trial`: 

In [6]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 1
Best trial:
  Value: 0.5135
  Params: 
    n_hidden: 1019
    learning_rate: 0.038747571156293785


### Saving your best model


When you're done with your hyperparameter search and have achieved at least 50% validation accuracy, you can save your best model to the `./models`-directory in order to submit the model.

Before that, we will check again whether the number of parameters is below 5 Mio and the file size is below 20 MB.

When your final model is saved, we'll lastly report the test accuracy.

In [7]:
best_model = load_model('checkpoints/{}.p'.format(best_trial.number))
best_model.prepare_data()
test_and_save(best_model)

Validation-Accuracy: 0.5035%
FYI: Your model has 3.266 params.
Great! Your model size is less than 20 MB and will be accepted :)
Your model has been saved and is ready to be submitted. NOW, let's check the test-accuracy.
Test-Accuracy: 0.4992%


### Remove checkpoints directory
Lastly, let's remove the checkpoints directory again to clear up space.

In [47]:
shutil.rmtree("checkpoints")

### References

Here we provided just a quick overview about hyperparameter optimization using Optuna. Optuna has a lot more to offer, so definitely make sure to check out the following resources:

* Source Code: https://github.com/optuna/optuna
* Docs: https://optuna.readthedocs.io/en/stable/
* Website: https://optuna.org
* Paper: https://arxiv.org/pdf/1907.10902.pdf